Trap Depth Calculations

In [38]:
import numpy as np
import scipy.constants as sc
from typing import Union

In [3]:
def to_freq_ang(wavelength: float) -> float:
    """Converts a wavelength to its angular frequency using the formula 
       omega = 2*pi*c / lambda

    Args:
        wavelength (float): Wavelength [nm]

    Returns:
        float: Angular frequency in s^-1
    """

    # Converts a wavelength in nanometers of light to radians
    return (2*np.pi*sc.speed_of_light*1e9)/(wavelength)

In [4]:
def alpha(wavelength: float) -> complex:
    """Calculates the polarizability alpha using equation (8) from the Grimm review paper

    Args:
        wavelength (float): wavelength [nm]

    Returns:
        complex: Complex polarizability alpha
    """
    # Units = (A^2s^4)/(kg rad)
    # Use Equation (8)

    # https://jet.physics.ncsu.edu/techdocs/pdf/PropertiesOfLi.pdf

    omega_0       = to_freq_ang(wavelength = 671)
    omega         = to_freq_ang(wavelength = wavelength)
    gamma_omega_0 = 36.898e6  
    # Line width of the transition (We are only interested in the damping there)

    result  = 6 * np.pi * sc.epsilon_0 * (sc.speed_of_light**3)
    result *= gamma_omega_0 / (omega_0**2)
    result /= complex(omega_0**2 - omega**2, - (omega**3/omega_0**2)*gamma_omega_0)

    return result


In [5]:
def potential(intensity: float, wavelength: float) -> float:
    """Calculates the dipole potential using equation (2) of the Grimms Review Paper

    Args:
        intensity (float): Intensity   [W/m^2]
        wavelength (float): Wavelength [nm]

    Returns:
        float: Potential 
    """
    
    # Equation (2) of Grimm
    _alpha = alpha(wavelength = wavelength)
    print("alpha =", _alpha)

    U_dip  = - np.real(_alpha) * intensity
    U_dip /= 2 * sc.epsilon_0 * sc.speed_of_light

    return U_dip

In [6]:
def trap_temperature(trap_depth: float) -> float:
    """Converts a trap depth (i.e. min dipole potential) to a temperature using T = U/kB

    Args:
        trap_depth (float): Trap Depth [J]

    Returns:
        float: Temperature of the trap
    """
    
    return -trap_depth/sc.Boltzmann

In [39]:
from typing import Tuple

def gaussian_beam_width(z: Union[float, np.ndarray], w_0: float , z_0: float, Msq: float, wavelength: float) -> Union[float, np.ndarray]:
    """ Returns the gaussian beam width based on a gaussian beam propagation
    
    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)


    Args:
        z (Union[float, np.ndarray]): Position in propagation direction [m, mm]
        w_0 (float): Beam waist                                         [m, um]
        z_0 (float): Position of beam waist                             [m, mm]
        Msq (float): M-squared beam quality factor                      [no unit]
        wavelength (float): Wavelength of light                         [m, nm]

    Returns:
        float: _description_
    """

    return w_0 * np.sqrt(
		1 + ((z - z_0)**2)*((
			(Msq * wavelength)/
			(np.pi * (w_0**2))
		)**2)
	)

def max_intensity(power: float, width_x: Union[float, np.ndarray], width_y: Union[float, np.ndarray]) -> Union[float, np.ndarray]:
    """Generate the max-intensity of the gaussian intensity distribution given power, w_x and w_y

    Function is normalized if:
    - Everything in SI-units, or
    - P [W], w_x [um], w_y [um] (preferred)

    Args:
        power (float): Power                      [W]
        width_x (Union[float, np.ndarray]): width [m, um]
        width_y (Union[float, np.ndarray]): width [m, um]

    Returns:
        float: Max intensity I_0 in [W/m^2, W/um^2]
    """
    return (2 * power) / (np.pi * (width_x*width_y))

def intensity(
        x: Union[float, np.ndarray], y: Union[float, np.ndarray], z: Union[float, np.ndarray],
        power: float, 
        wavelength: float,
        w_0: Tuple[float, float],
        z_0: Tuple[float, float],
        Msq: Tuple[float, float]
    ) -> Union[float, np.ndarray]:

    """Returns the intensity of a guassian beam at a point in 3D space. Assumes a simple astigmatic beam

    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)

    Args:
        x (Union[float, np.ndarray]): x position (one of the main axes)     [m, um]
        y (Union[float, np.ndarray]): y position (one of the main axes)     [m, um]
        z (Union[float, np.ndarray]): z position (propagation direction)    [m, mm]
        power (float): Power of the beam                                    [W]
        wavelength (float): Wavelength of the light                         [m, nm]
        w_0 (Tuple[float, float]): Tuple of the beam waist (x, y axes)      [m, um]
        z_0 (Tuple[float, float]): Tuple of the rayleigh length (x, y axes) [m, mm]
        Msq (Tuple[float, float]): Tuple of the beam quality factor M^2 (x, y axes)

    Returns:
        float: The intensity at that point [W/m^2, W/um^2]
    """ 

    # the x and y are the main axes for astigmatism

    x_params = { "w_0": w_0[0], "z_0": z_0[0], "Msq": Msq[0], "wavelength": wavelength }
    y_params = { "w_0": w_0[1], "z_0": z_0[1], "Msq": Msq[1], "wavelength": wavelength }

    w_x = gaussian_beam_width(z = z, **x_params)
    w_y = gaussian_beam_width(z = z, **y_params)

    I0 = max_intensity(power = power, width_x = w_x, width_y = w_y)

    intensity = I0 * np.exp(-2*((x/w_x)**2 + (y/w_y)**2))

    return intensity

# Check units

In [8]:
x = gaussian_beam_width(z = 3,    w_0 = 5, z_0 = 6, Msq = 1, wavelength = 1070)
y = gaussian_beam_width(z = 3e-3, w_0 = 5e-6, z_0 = 6e-3, Msq = 1, wavelength = 1070e-9)

print(x, y * 1e6) # convert to um

204.4161058594956 204.41610585949562


In [9]:
x = max_intensity(power = 100, width_x = 60, width_y = 60)
y = max_intensity(power = 100, width_x = 40e-6, width_y = 40e-6)
print(x, y * 1e-12) # convert to W/um^2

0.01768388256576615 0.03978873577297383


# Calculations

In [10]:
U = potential(intensity = x*1e12, wavelength = 1070) # SI UNIT OUTPUT
temp = trap_temperature(U)

print(U, "J", temp*1e3, "mK")

alpha = (4.403504782680971e-39+2.352531047837446e-47j)
-1.466819470370436e-26 J 1.0624130176246358 mK


# Combining multiple Gaussians

In [11]:
from scipy.spatial.transform import Rotation

def transform_quaternion_angle(axis: np.ndarray, degrees: float) -> Rotation: 
    """Generates a scipy.spatial.transform.Rotation object from a rotation axis and how many degrees to rotate.
    The Rotation object is generated using quarternions

    q = cos(a/2) + sin(a/2)(x*i + y*j + z*k)

    Args:
        axis (np.ndarray): Rotation axis in the form of (1,3) or (3,1) np array
        degrees (float): Degrees to rotate

    Returns:
        Rotation: scipy.spatial.transform.Rotation object
    """

    halfangle = np.deg2rad(degrees)/2

    unit_axis = axis.flatten()
    unit_axis = unit_axis / np.linalg.norm(unit_axis)

    ax = np.sin(halfangle) * unit_axis
    q  = np.concatenate(([np.cos(halfangle)], ax))

    return Rotation.from_quat(q)

def rotate_points(points: np.ndarray, axis: np.ndarray, degrees: float) -> np.ndarray:
    """Rotate a point or multiple points by rotating them

    Args:
        points (np.ndarray): Points to be transformed
        axis (np.ndarray): Rotation axis in the form of (1,3) or (3,1) np array
        degrees (float): Degrees to rotate

    Returns:
        np.ndarray: Transformed points
    """
    _R  = transform_quaternion_angle(axis = axis, degrees = degrees)
    _xp = _R.apply(points)

    return _xp

In [12]:
x    = np.array([0, 0, 3])
axis = np.array([0, 1, 0]) # y-axis
xp = rotate_points(points = x, axis = axis, degrees = 90)

print(x, xp)

[0 0 3] [3. 0. 0.]


# Generate potential at every point

In [34]:
beam_params = [
    {
        "w_0": [25  , 25 ], # um
        "z_0": [0   , 0  ], # mm
        "Msq": [1.1 , 1.1],
        "wavelength": 1070  #nm
    },
    {
        "w_0": [25  , 25 ], # um
        "z_0": [0   , 0  ], # mm
        "Msq": [1.1 , 1.1],
        "wavelength": 1070  #nm
    }
]

# We generate in the x-z plane

x = np.linspace(start = -50, stop = 50, num = 1000, endpoint = True) # um
z = np.linspace(start = -5, stop = 5, num = 1000, endpoint = True) # mm
y = np.zeros(shape = x.shape)

# https://stackoverflow.com/a/44409182
points = np.column_stack((x,y))
points = np.column_stack((points, z))

rotation_axis = np.array([0,1,0]) # y-axis
angle_between_beams = 15
power = 100 # W

points_beam1 = rotate_points(points = points, axis = rotation_axis, degrees =  angle_between_beams/2)
points_beam2 = rotate_points(points = points, axis = rotation_axis, degrees = -angle_between_beams/2)

In [40]:
intensities_1 = intensity(
    x = points_beam1[:,0], y = points_beam1[:,1], z = points_beam1[:,2], 
    power = power,
    **beam_params[0]
)
intensities_2 = intensity(
    x = points_beam2[:,0], y = points_beam2[:,1], z = points_beam2[:,2], 
    power = power,
    **beam_params[1]
)

print(intensities_2)

[0.00179701 0.0018041  0.00181123 0.00181841 0.00182562 0.00183288
 0.00184019 0.00184753 0.00185492 0.00186236 0.00186984 0.00187736
 0.00188493 0.00189254 0.00190021 0.00190791 0.00191567 0.00192347
 0.00193131 0.00193921 0.00194716 0.00195515 0.00196319 0.00197128
 0.00197942 0.00198761 0.00199586 0.00200415 0.00201249 0.00202089
 0.00202934 0.00203784 0.00204639 0.002055   0.00206366 0.00207238
 0.00208115 0.00208998 0.00209886 0.0021078  0.0021168  0.00212585
 0.00213496 0.00214413 0.00215336 0.00216265 0.00217199 0.0021814
 0.00219087 0.0022004  0.00220999 0.00221965 0.00222936 0.00223914
 0.00224899 0.0022589  0.00226887 0.00227891 0.00228902 0.00229919
 0.00230943 0.00231974 0.00233012 0.00234056 0.00235108 0.00236166
 0.00237232 0.00238305 0.00239385 0.00240473 0.00241568 0.0024267
 0.0024378  0.00244897 0.00246022 0.00247155 0.00248295 0.00249444
 0.002506   0.00251764 0.00252936 0.00254117 0.00255306 0.00256503
 0.00257708 0.00258922 0.00260144 0.00261375 0.00262614 0.002638